In [2]:
import sys
!pip install PyQt5
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtSql import QSqlDatabase, QSqlTableModel
from PyQt5.QtWidgets import (
    QApplication,
    QComboBox,
    QDataWidgetMapper,
    QDateTimeEdit,
    QFormLayout,
    QHBoxLayout,
    QLabel,
    QLineEdit,
    QMainWindow,
    QPushButton,
    QVBoxLayout,
    QWidget,
)


In [3]:
db = QSqlDatabase("QSQLITE")
db.setDatabaseName("members.sqlite")
# Create some dummy data.
db.open()
db.exec_(
    "CREATE table if not exists members(name string, experience string, skills string, open_to_work string, portfolio_site string, created datetime);"
)
db.exec_(
    "insert into members(name, experience, skills, open_to_work, portfolio_site, created) values('Name', 'Level of Experience', 'Top 3 Skills', 'Open to Receiving Job Leads?', 'Portfolio or Linkedin' datetime('Today') );"
)
#db.exec_(
    #"insert into members (title, experience, created) values ('2nd item', 'Three', datetime('2019-06-02 12:45:11') );"
#)
# SQL script to create the 'members' table
#create_table_query = '''
#CREATE TABLE IF NOT EXISTS members (
    #FirstName VARCHAR(50),
    #LastName VARCHAR(50),
    #Portfolio VARCHAR(100),
    #Resume TEXT,
    #ExperienceLevel INT,
    #Skills TEXT,
    #OpenToJobs BOOLEAN,
    #created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
#);
#'''

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        form = QFormLayout()

        # Define fields.
        self.first_name = QLineEdit()
        self.last_name = QLineEdit()
        self.social = QLineEdit()
        self.experience = QComboBox()
        self.experience.addItems(["Just Starting", "Two(2)-Five(5) Years", "More than 5 Years"])
        self.skills = QLineEdit()
        self.date = QDateTimeEdit()
        #self.date.setDisplayFormat("yyyy-MM-dd HH:mm:ss")
        self.work = QComboBox()
        self.work.addItems(["Yes", "No"])

        form.addRow(QLabel("First Name"), self.first_name)
        form.addRow(QLabel("Last Name"), self.last_name)
        form.addRow(QLabel("Website: Portfolio/Linkedin"), self.social)
        form.addRow(QLabel("Level of Experience"), self.experience)
        form.addRow(QLabel("Top 3 Skills"), self.skills)
        form.addRow(QLabel("Open to Receiving Job Leads?"), self.work)
        form.addRow(QLabel("Date"), self.date)

        self.model = QSqlTableModel(db=db)

        self.mapper = QDataWidgetMapper()  # Syncs widgets to the database.
        self.mapper.setModel(self.model)

        self.mapper.addMapping(self.first_name, 0) # Map to column number
        self.mapper.addMapping(self.last_name, 1) 
        self.mapper.addMapping(self.social, 2)
        self.mapper.addMapping(self.experience, 3)
        self.mapper.addMapping(self.skills, 4)
        self.mapper.addMapping(self.work, 5)
        self.mapper.addMapping(self.date, 6)

        self.model.setTable("members")
        self.model.select()  # Query the database

        self.mapper.toFirst()  # Jump to first record

        self.setMinimumSize(QSize(400, 400))

        controls = QHBoxLayout()

        prev_rec = QPushButton("Previous")
        prev_rec.clicked.connect(self.mapper.toPrevious)

        #next_rec = QPushButton("Next")
        #next_rec.clicked.connect(self.mapper.toNext)

        save_rec = QPushButton("Save Changes")
        save_rec.clicked.connect(self.mapper.submit)
        
        #done_rec = QPushButton("Done")
        #done_rec.clicked.connect(self.mapper.)

        controls.addWidget(prev_rec)
        #controls.addWidget(next_rec)
        controls.addWidget(save_rec)

        layout = QVBoxLayout()

        layout.addLayout(form)
        layout.addLayout(controls)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)


app = QApplication(sys.argv)
window = MainWindow()
window.show()
app.exec_()

0